# The Sed object

This example demonstrates the various methods associated with the `Sed` class.

`Sed` objects can be extracted directly from `Grid` objects or created by `Galaxy` objects. See tutorials on those objects for more information.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import cmasher as cmr
import os
from unyt import Angstrom, Hz, um
from synthesizer.grid import Grid, get_available_lines
from synthesizer.sed import Sed
from synthesizer.filters import FilterCollection
from synthesizer.igm import Madau96

Let's begin by initialising a grid:

In [ ]:

grid_dir =  '../../tests/test_grid/'
grid_name = 'test_grid'
grid = Grid(grid_name, grid_dir=grid_dir)

Next, let's define a target log10age and metallicity and use the built-in `Grid` method to get the grid point and then extract the spectrum for that grid point.

In [ ]:
log10age = 6. # log10(age/yr)
metallicity = 0.01 
spectra_id = 'incident'
grid_point = grid.get_grid_point((log10age, metallicity))
sed = grid.get_spectra(grid_point, spectra_id=spectra_id)
sed.lnu *= 1E8 # make the SED bigger

Like other `synthesizer` objects, we get some basic information about the `Sed` object by using the `print` command:

In [ ]:
print(sed)

`Sed` objects contain a wavelength grid and luminosity in the `lam` and `lnu` attributes. Both come with units making them easy to convert:

In [ ]:
print(sed.lam)
print(sed.lnu)

Thus we can easily make a plot:

In [ ]:
plt.plot(np.log10(sed.lam), np.log10(sed.lnu))
plt.xlabel(r"$\rm log_{10}(\lambda/\AA)$")
plt.ylabel(r"$\rm log_{10}(L_{\nu}/erg\ s^{-1}\ Hz^{-1} M_{\odot}^{-1})$")
plt.show()
plt.close()

### Methods

#### get_bolometric_luminosity()

This method allows us to calculate the bolometric luminosity of the sed. 

In [ ]:
sed.measure_bolometric_luminosity()

By default the above simply sums up the spectrum. However, we can also integrate the spectrum instead:

In [ ]:
sed.measure_bolometric_luminosity(method='quad')

Under-the-hood the above uses a function to get the luminosity at a particular frequency or wavelength:

In [ ]:
sed.get_lnu_at_lam(1500 * Angstrom)

In [ ]:
sed.get_lnu_at_nu(1E14 * Hz)

In [ ]:
sed.measure_window_luminosity((1400.*Angstrom,1600.*Angstrom))

In [ ]:
sed.measure_window_luminosity((0.14*um,0.16*um))

In [ ]:
sed.measure_window_lnu((1400.*Angstrom,1600.*Angstrom))

In [ ]:
sed.measure_window_lnu((1400.*Angstrom,1600.*Angstrom), method='average')

In [ ]:
sed.measure_window_lnu((1400, 1600)*Angstrom, method='quad')

We can also measure a spectral break by providing two windows, e.g.

In [ ]:
sed.measure_break((3400, 3600) * Angstrom, (4150, 4250) * Angstrom)

There are also a few in-built break methods, e.g. `measure_Balmer_break()`

In [ ]:
sed.measure_balmer_break()

In [ ]:
sed.measure_d4000()

We can also measure absorption line indices:

In [ ]:
sed.measure_index((1500,1600)*Angstrom, (1400,1500)*Angstrom, (1600,1700)*Angstrom)

We can also measure the UV spectral slope $\beta$:

In [ ]:
sed.measure_beta()

By default this uses a single window and fits the spectrum by a power-law. However, we can also specify two windows as below, in which case the luminosity in each window is calcualted and used to infer a slope:

In [ ]:
sed.measure_beta(window=(1250,1750,2250,2750))

## Observed frame SED

To do this we need to provide a cosmology, using an `astropy.cosmology` object, a redshift $z$, and optionally an IGM absorption model.

In [ ]:
from astropy.cosmology import Planck18 as cosmo
z = 10.  # redshift
sed.get_fnu(cosmo, z, igm=Madau96)  # generate observed frame spectra

## Broadband fluxes

Once we have computed the observed frame SED there is a method on an `Sed` object that allows us to calculate broadband fluxes. However, first we need to instantiate a `FilterCollection` object.

In [ ]:
filter_codes = [f'JWST/NIRCam.{f}' for f in ['F070W','F090W', 'F115W', 'F150W',
                                             'F200W', 'F277W', 'F356W', 'F444W']]  # define a list of filter codes
fc = FilterCollection(filter_codes, new_lam=grid.lam)

In [ ]:
# measure broadband fluxes
fluxes = sed.get_broadband_fluxes(fc)

# print broadband fluxes
for filter, flux in fluxes.items():
    print(f'{filter}: {flux:.2f}')

## Multiple SEDs

The `Sed` object can actually hold an array of seds and the methods should all work fine.

Let's create an `Sed` object with two seds:

In [ ]:
sed2 = Sed(sed.lam, np.array([sed.lnu, sed.lnu * 2]))

In [ ]:
sed2.measure_window_lnu((1400,1600)*Angstrom)

In [ ]:
sed2.measure_window_lnu((1400,1600)*Angstrom, method='average')

In [ ]:
sed2.measure_beta()

In [ ]:
sed2.measure_beta(window=(1250,1750,2250,2750))

In [ ]:
sed2.measure_balmer_break()

In [ ]:
sed2.measure_index((1500,1600)*Angstrom, (1400,1500)*Angstrom, (1600,1700)*Angstrom)

### Combining SEDs

`Sed`s can be combined either via concatenation to produce a single `Sed` holding multiple spectra from the combined `Sed`s, or by addition to add the spectra contained in two `Sed`s. 

To concatenate spectra we can use `Sed.concat()`.

In [ ]:
print("Shapes before:", sed._lnu.shape, sed2._lnu.shape)
sed3 = sed2.concat(sed)
print("Combined shape:", sed3._lnu.shape)

`Sed.concat` can take an arbitrary number of `Sed` objects to combine.

In [ ]:
sed4 = sed2.concat(sed, sed2, sed3)
print("Combined shape:", sed4._lnu.shape)

If we want to add the spectra of 2 `Sed` objects we simply apply the `+` operator. However, unlike `concat`, this will only work for `Sed`s with identical shapes.

In [ ]:
sed5 = sed + sed
plt.plot(np.log10(sed.lam), np.log10(sed.lnu), label="sed")
plt.plot(np.log10(sed5.lam), np.log10(sed5.lnu), label="sed5")
plt.ylim(26, 30)
plt.xlim(2, 5)
plt.xlabel(r"$\rm log_{10}(\lambda/\AA)$")
plt.ylabel(r"$\rm log_{10}(L_{\nu}/erg\ s^{-1}\ Hz^{-1} M_{\odot}^{-1})$")
plt.legend()
plt.show()
plt.close()

## Resampling SEDs

The `Sed` includes a method to resample an sed, e.g. to lower-resolution or to match observations.

In [ ]:
sed6 = sed.get_resampled_sed(5)
plt.plot(np.log10(sed.lam), np.log10(sed.lnu), label="Original")
plt.plot(np.log10(sed6.lam), np.log10(sed6.lnu), label="Resampled")
plt.xlim(2.2, 3.5)
plt.ylim(27., 29.5)
plt.xlabel(r"$\rm log_{10}(\lambda/\AA)$")
plt.ylabel(r"$\rm log_{10}(L_{\nu}/erg\ s^{-1}\ Hz^{-1} M_{\odot}^{-1})$")
plt.legend()
plt.show()
plt.close()

In [ ]:
print(sed.measure_bolometric_luminosity()/sed3.measure_bolometric_luminosity())